In [ ]:
#导入相关库
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
#加载训练数据集（数据量太大，分块载入）、测试数据示例、提交结果示例
pd_train=pd.read_csv('../input/ubiquant-market-prediction/train.csv',iterator=True)#生成迭代器
pd_train_chunk=pd.read_csv('../input/ubiquant-market-prediction/train.csv',chunksize=100000)#设定每块大小为（100000，304）
pd_test_exm=pd.read_csv('../input/ubiquant-market-prediction/example_test.csv')
pd_sub_exm=pd.read_csv('../input/ubiquant-market-prediction/example_sample_submission.csv')
print('done')

In [ ]:
#训练数据示例
display(pd_train.get_chunk(10))

In [ ]:
#观察训练数据量大小,#训练数据集大小：(3141410,304)
#同步观察是否存在缺失值，无缺失。
'''
all_row_num=[]
i=0
for chunk in pd_train_chunk:
    i+=1
    shape=chunk.shape
    row_num=shape[0]
    all_row_num.append(row_num)
    if chunk.isnull().sum().sum()!=0:
        print('第{}块存在缺失值'.format(i))
    else:
        print(i)
print('done')
print('训练数据集大小：({},{})'.format(sum(all_row_num),shape[1]))
'''

**观察row_id、time_id、investment_id、target的数据分布**

In [ ]:
#仅加载训练数据前四列
#pd_train_id=pd.read_csv('../input/ubiquant-market-prediction/train.csv',usecols=['row_id','time_id','investment_id','target'])

In [ ]:
#row_id是主键，且满足time_id+"_"+investment_id==row_id
'''
print('row_id是否唯一：',pd_train_id['row_id'].nunique()==pd_train_id['row_id'].count())
pd_train_id['row_id_1']=[str(i)+"_"+str(j) for i,j in zip(pd_train_id['time_id'],pd_train_id['investment_id'])]
print('验证time_id+"_"+investment_id==row_id：',pd_train_id.query('row_id!=row_id_1')['row_id'].count()==0)

#time_id并不连续的，存在中断的情况。中断的不存在的时间节点有：361,368,369,370,371,372,382,402,403。若每个time_id为一天，则有共计有1211天，3.5年左右。
#investment_id并不是连续的，而是在同一个time_id内自增，且自增也不连续。共计存在3579项投资。
print('--------------------------------------------------')
print('time_id的最小值为：{}，最大值为:{}，不重复数量为：{}'.format(pd_train_id['time_id'].min(),pd_train_id['time_id'].max(),pd_train_id['time_id'].nunique()))
print('investment_id的最小值为：{}，最大值为:{}，不重复数量为：{}'.format(pd_train_id['investment_id'].min(),pd_train_id['investment_id'].max(),pd_train_id['investment_id'].nunique()))
print('--------------------------------------------------')
print('time_id不存在的时间节点有：')
row_id_unique=list(pd_train_id['time_id'].unique())
for i in range(max(row_id_unique)+1):
    try:
        row_id_unique.index(i)
    except:
        print(i)

        
#time_id均值：665.49         方差：356.04
#investment_id均值1885.27   方差：1083.62
print('--------------------------------------------------')
print('time_id的均值为：{}\ntime_id的方差为：{}'.format(round(pd_train_id['time_id'].mean(),2),round(pd_train_id['time_id'].std(),2)))
print('\ninvestment_id的均值为：{}\ninvestment_id的方差为：{}'.format(round(pd_train_id['investment_id'].mean(),2),round(pd_train_id['investment_id'].std(),2)))
'''

In [ ]:
#观察time_id每天的记录数分布。
#随着时间的推移呈递增缓慢趋势，在200-600间呈现较明显的波动（time_id缺失节点：361,368,369,370,371,372,382,402,403）
#每天对应的投资项数量逐渐增多，投资项数量呈现正态分布。
'''
time_id_size=pd_train_id.groupby('time_id').size().to_frame().reset_index().rename(columns={0:"count"})
print('记录数最少的一天是：{},共有记录数：{}'.format(time_id_size.sort_values('count').iloc[0][0],time_id_size.sort_values('count').iloc[0][1]))
print('记录数最多的一天是：{},共有记录数：{}'.format(time_id_size.sort_values('count').iloc[-1][0],time_id_size.sort_values('count').iloc[-1][1]))
print('平均记录数是：',round(time_id_size['count'].mean(),2))
print('\n')
#绘制趋势图和分布图
plt.style.use('_mpl-gallery')
plt.figure(figsize=(8,6),dpi=80)
ax=plt.subplot(211,xlabel="time_id",ylabel='count')
ax.plot(time_id_size['time_id'],time_id_size['count'],linewidth=0.8,color='red')
ax=plt.subplot(212,xlabel="time_count",ylabel='num')
hist=ax.hist(time_id_size['count'],bins=80,linewidth=1,edgecolor="white")


#观察investment_id每项投资出现的次数（天数）分布。
#每项投资的次数呈现右偏分布，说明投资次数较多的投资项的占大多数。
invest_id_size=pd_train_id.groupby('investment_id').size().to_frame().reset_index().rename(columns={0:"count"})
print('出现次数(天数)最少的投资项是：{}，共出现多少次数(天数)：{}'.format(invest_id_size.sort_values('count').iloc[0][0],invest_id_size.sort_values('count').iloc[0][1]))
print('出现次数(天数)最多的投资项是：{}，共出现多少次数(天数)：{}'.format(invest_id_size.sort_values('count').iloc[-1][0],invest_id_size.sort_values('count').iloc[-1][1]))
print('平均记录数是：',round(invest_id_size['count'].mean(),2))
#绘制分布图
plt.figure(figsize=(8,4),dpi=80)
plt.hist(invest_id_size['count'],bins=80,linewidth=1,edgecolor="white")
plt.xlabel('investment_count')
plt.ylabel('num')
'''

In [ ]:
#观察target，随着时间time_id的分布。
#按time_id聚合求target_sum。（可切换求target_mean）
#其趋势围绕0上下波动，在400处波动剧烈。taget_sum呈现正态分布（尖峰）。均值小于0。
'''
time_id_target=pd_train_id.groupby('time_id').agg({'target':'sum'}).reset_index()
print('target_sum最少的一天是：{},共计：{}'.format(int(time_id_target.sort_values('target').iloc[0][0]),time_id_target.sort_values('target').iloc[0][1]))
print('target_sum最多的一天是：{},共计：{}'.format(int(time_id_target.sort_values('target').iloc[-1][0]),time_id_target.sort_values('target').iloc[-1][1]))
print('平均target_sum是：',round(time_id_target['target'].mean(),2))
print('\n')
#绘制趋势图和分布图
plt.figure(figsize=(8,6),dpi=80)
ax=plt.subplot(211,xlabel="time_id",ylabel="target_sum")
ax.plot(time_id_target['time_id'],time_id_target['target'],linewidth=0.8,color='red')
ax=plt.subplot(212,xlabel="target_sum",ylabel="num")
hist=ax.hist(time_id_target['target'],bins=80,linewidth=1,edgecolor="white")


#观察target，关于investment_id的分布。
#按investment_id聚合求target_sum。（可切换求target_mean）
#其趋势围绕0上下波动。taget_sum呈现正态分布（矮峰）。均值小于0。
invest_id_target=pd_train_id.groupby('investment_id').agg({'target':'sum'}).reset_index()
print('target_sum最少的投资项是：{},共计：{}'.format(int(invest_id_target.sort_values('target').iloc[0][0]),invest_id_target.sort_values('target').iloc[0][1]))
print('target_sum最多的投资项是：{},共计：{}'.format(int(invest_id_target.sort_values('target').iloc[-1][0]),invest_id_target.sort_values('target').iloc[-1][1]))
print('平均target_sum是：',round(invest_id_target['target'].mean(),2))
print('\n')
#绘制趋势图和分布图
plt.figure(figsize=(8,6),dpi=80)
ax=plt.subplot(211,xlabel="invest_id",ylabel="target_sum")
ax.plot(invest_id_target['investment_id'],invest_id_target['target'],linewidth=0.8,color='red')
ax=plt.subplot(212,xlabel="target_sum",ylabel="num")
hist=ax.hist(invest_id_target['target'],bins=80,linewidth=1,edgecolor="white")
'''

In [ ]:
#观察f_0~f_299的数据分布
'''
pd_train_chunk=pd.read_csv('../input/ubiquant-market-prediction/train.csv',chunksize=100000)
f_min=pd.DataFrame(columns=['f_'+str(i) for i in range(300)])
f_max=pd.DataFrame(columns=['f_'+str(i) for i in range(300)])
for id,chunk in enumerate(pd_train_chunk):
    print('第{}个chunk,共计{}行数据'.format(id,(id+1)*100000))
    f_max.loc[id]=chunk.iloc[:,4:].max().tolist()
    f_min.loc[id]=chunk.iloc[:,4:].min().tolist()
    
#f_0~f_299的最大值和最小值为80,-80
f_max_all=f_max.max()
f_min_all=f_min.min()
#绘制图形
plt.style.use('_mpl-gallery')
plt.figure(figsize=(8,6),dpi=80)
ax=plt.subplot(211,xlabel="columns",ylabel='max_value')
ax.plot(f_max_all,linewidth=0.8,color='red')
ax=plt.subplot(212,xlabel="columns",ylabel='min_value')
ax.plot(f_min_all,linewidth=0.8,color='red')
'''

**目标编码**

In [ ]:
'''
#进行目标编码，对investment_id进行10折目标均值编码，测试集则使用训练集所有数据进行均值编码。
k=10
all_row_num=3141410
pd_train_id['fold']=[i for i in range(k) for j in range(int(all_row_num/k))]#标记K折
pd_fold=pd.DataFrame(columns=['investment_id','fold','invest_target'])#创建K折均值编码数据框（包含训练集所有均值编码，为-1）

#先导入训练集所有均值编码，fold标记为-1
pd_fold_all=pd_train_id.groupby('investment_id').agg({'target':'mean'}).reset_index().rename(columns={'target':'invest_target'})
pd_fold_all['fold']=-1
pd_fold=pd_fold.append(pd_fold_all,ignore_index=True)

#依次计算出所有折的均值编码，并记录在pd_fold中
for i in range(k):
    pd_fold_i=pd_train_id.query('fold!='+str(i)).groupby('investment_id').agg({'target':'mean'}).reset_index().rename(columns={'target':'invest_target'})
    pd_fold_i['fold']=i
    pd_fold=pd_fold.append(pd_fold_i,ignore_index=True)
print('展示部分均值编码后的数据：')
display(pd_fold.head(5))

#训练集中的investment_id不重复数量为3579，观察K折均值目标编码后，每组中的数量。存在部分组中的investment_id数量不全.
print('\n观察K折均值目标编码后，每组中的数量:')
display(pd_fold.groupby('fold').size().to_frame().reset_index().rename(columns={0:'num'}))

#将编码后的数据与原数据进行匹配并赋值,若未匹配到则置为0
pd_train_id=pd_train_id.merge(pd_fold,on=['investment_id','fold'],how='left').fillna(0)
print('\n展示部分匹配均值编码后的数据：')
display(pd_train_id.head(5))

#对K折编码数据进行保存，以便于下次调用
pd_fold.to_csv('./pd_fold.csv',index=None)

#对已编码赋值后的数据保存，便于下次调用（直接添加为最后一列）
pd_train_id[['row_id','invest_target']].to_csv('./pd_invest_target.csv',index=None)
'''

**划分训练集和测试集**

In [ ]:
'''
#划分训练集和测试集，训练：测试=9:1
#未shuffle,选取后10%作为测试集，训练集约为280W，测试级约为30W。共计314W
all_num=3141410
train_num=int(all_num*0.9)
test_num=int(all_num-train_num)

#加载训练集和测试集数据
pd_train=pd.read_csv('../input/ubiquant-market-prediction/train.csv',iterator=True)
pd_test=pd.read_csv('../input/ubiquant-market-prediction/train.csv',skiprows=train_num,header=0,names=['row_id','time_id','investment_id','target']+['f_'+str(i) for i in range(300)])
#加载K折编码数据
pd_fold=pd.read_csv('../input/pd-fold/pd_fold.csv')
pd_invest_target=pd.read_csv('../input/pd-invest-target/pd_invest_target.csv')
'''

In [ ]:
'''
#构造输入特征和目标值，选择f_0~f_299作为特征变量，target为目标值
def train_generator(batch=200,pd_train=pd_train):
    while True:
        pd_train_chunk=pd_train.get_chunk(batch)
        #对time_id和investment_id进行标准化
        pd_train_chunk['time_id']=(pd_train_chunk['time_id']-665.49)/356.04
        pd_train_chunk['investment_id']=(pd_train_chunk['investment_id']-1885.27)/1083.62
        #增加目标均值编码特征
        pd_train_chunk['invest_target']=pd_invest_target.iloc[pd_train_chunk.index[0]:(pd_train_chunk.index[-1]+1)]['invest_target']
        #获取特征值和目标变量
        feature=pd_train_chunk.drop(['row_id','target','time_id','investment_id','invest_target'],axis=1).to_numpy()
        target=pd_train_chunk['target'].to_numpy().reshape(-1,1)
        yield (feature,target)
        if pd_train_chunk.index[-1]>=(train_num-batch):
            pd_train=pd.read_csv('../input/ubiquant-market-prediction/train.csv',iterator=True)
        else:
            pass

def test_generator(batch=1000,pd_test=pd_test):
    ##对time_id和investment_id进行标准化
    pd_test['time_id']=(pd_test['time_id']-665.49)/356.04
    pd_test['investment_id']=(pd_test['investment_id']-1885.27)/1083.62
    #增加目标均值编码特征
    pd_test['invest_target']=pd_invest_target.iloc[train_num:,:]['invest_target']
    #获取特征值和目标变量
    i=0
    while True:
        if i>=test_num:
            i=0
        else:
            pass
        pd_test_chunk=pd_test.iloc[i:i+batch,:]
        feature=pd_test_chunk.drop(['row_id','target','time_id','investment_id','invest_target'],axis=1).to_numpy()
        target=pd_test_chunk['target'].to_numpy().reshape(-1,1)
        i+=batch
        yield (feature,target)
'''

In [ ]:
'''
#构建模型
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras import models,layers,optimizers,regularizers

#刷新训练数据
pd_train=pd.read_csv('../input/ubiquant-market-prediction/train.csv',iterator=True)

#构建模型
model=models.Sequential([
    layers.Dense(128,activation='relu',input_shape=(300,)),
    layers.Dense(64,activation='relu'),
    layers.Dense(32,activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer=optimizers.Adam(0.001),
              loss='mse',
              metrics=['mse'])

model.fit(train_generator(),epochs=3,steps_per_epoch=14000,validation_data=test_generator(),validation_steps=300)
'''

In [ ]:
#model.fit(train_generator(),epochs=1,steps_per_epoch=14000,validation_data=test_generator(),validation_steps=300)

In [ ]:
#保存模型
#model.save('./model.h5')

In [ ]:
#进行预测
import pandas as pd
from tensorflow.keras.models import load_model
model=load_model('../input/model-11/model.h5')
#加载K折编码数据
pd_fold=pd.read_csv('../input/pd-fold/pd_fold.csv')
pd_fold=pd_fold.query('fold==-1')[['investment_id','invest_target']]

import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission


for (test_df, sample_prediction_df) in iter_test:
    #增加目标均值编码特征
    #test_df=test_df.merge(pd_fold,on='investment_id',how='left').fillna(0)
    #提取time_id
    #test_df['time_id']=test_df['row_id'].apply(lambda x : int(x[:x.index('_')]))
    ##对time_id和investment_id进行标准化
    #test_df['time_id']=(test_df['time_id']-665.49)/356.04
    #test_df['investment_id']=(test_df['investment_id']-1885.27)/1083.62
    #获取特征值和目标变量
    feature=test_df.drop(['row_id','investment_id'],axis=1).to_numpy()  
    pre=model.predict(feature)
    sample_prediction_df['target'] = pre  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions
    display(sample_prediction_df)